<a href="https://colab.research.google.com/github/sai-raman/face-mask-detection/blob/main/mask_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2,os
import keras
import numpy as np
from keras.utils import np_utils

from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split


In [ ]:

data_path='./drive/My Drive/colab_dataset/data'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels))


In [ ]:
labels
label_dict
categories

['with_mask', 'without_mask']

In [ ]:


data=[]
target=[]

for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
            resized=cv2.resize(gray,(100, 100))
            data.append(resized)
            target.append(label_dict[category])

        except Exception as e:
            print('Exception:',e)
            

In [ ]:
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],100, 100,1))
target=np.array(target)

In [ ]:

new_target=np_utils.to_categorical(target)

np.save('data',data)
np.save('target',new_target)

In [ ]:
data=np.load('data.npy')
target=np.load('target.npy')

model=Sequential()

model.add(Conv2D(100,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50,activation='relu'))
model.add(Dense(2,activation='softmax'))

In [ ]:

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

checkpoint = ModelCheckpoint(
  'mask_model_latest.h5',
  monitor='val_loss',
  verbose=0,
  save_best_only=True,
  mode='auto')

history=model.fit(
  train_data,
  train_target, 
  epochs=20,
  callbacks=[checkpoint],
  validation_split=0.2)

print(model.evaluate(test_data,test_target))

Epoch 1/20
31/31 [==============================] - 2s 74ms/step - loss: 0.8017 - accuracy: 0.5434 - val_loss: 0.5872 - val_accuracy: 0.7137
Epoch 2/20
31/31 [==============================] - 2s 55ms/step - loss: 0.5518 - accuracy: 0.7182 - val_loss: 0.4237 - val_accuracy: 0.8306
Epoch 3/20
31/31 [==============================] - 2s 54ms/step - loss: 0.3669 - accuracy: 0.8475 - val_loss: 0.2720 - val_accuracy: 0.9073
Epoch 4/20
31/31 [==============================] - 2s 55ms/step - loss: 0.2309 - accuracy: 0.9101 - val_loss: 0.2385 - val_accuracy: 0.9113
Epoch 5/20
31/31 [==============================] - 2s 55ms/step - loss: 0.1614 - accuracy: 0.9495 - val_loss: 0.1519 - val_accuracy: 0.9516
Epoch 6/20
31/31 [==============================] - 2s 53ms/step - loss: 0.1154 - accuracy: 0.9636 - val_loss: 0.1562 - val_accuracy: 0.9355
Epoch 7/20
31/31 [==============================] - 2s 52ms/step - loss: 0.1113 - accuracy: 0.9586 - val_loss: 0.2544 - val_accuracy: 0.9032
Epoch 8/20
31

In [ ]:
model.save('mask_model_saved.h5')